In [ ]:
import imageio
import random
import numpy as np  
import pandas as pd 

from cellpose import models
from cellpose import utils as cp_utils
from cellstitch.utils import *
from cellstitch.pipeline import *
from cellstitch.evaluation import *

In [ ]:
filenames = set(get_filenames("../DATA/ATAS/raw/"))

In [ ]:
# test_filenames = set(random.sample(filenames, int(0.3 * len(filenames))))
# train_filenames = filenames - test_filenames

### Train cellpose model from scratch

In [ ]:
# generate training data
data_folder = "../DATA/ATAS"
cellpose_folder = "../DATA/ATAS/cellpose_train"

In [ ]:
for train_filename in train_filenames: 
    img = np.load("%s/raw/%s" % (data_folder, train_filename))
    labels = np.load("%s/labels/%s" % (data_folder, train_filename)) 
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("%s/%s_%s.tif" % (cellpose_folder, train_filename, i), img[i])
        imageio.imwrite("%s/%s_%s_masks.tif" % (cellpose_folder, train_filename, i), labels[i])

`python -m cellpose --train --use_gpu --dir ../DATA/ATAS/cellpose_train --pretrained_model None --n_epochs 100  --verbose`

### Test file names

In [ ]:
filenames = set(get_filenames("../DATA/ATAS/raw/"))

In [ ]:
train_filenames = []

In [ ]:
for file in os.listdir("../DATA/ATAS/cellpose_train/"):
    if file.endswith(".npy_95.tif"):
        train_filenames.append(file[:-7])

In [ ]:
test_filenames = filenames - set(train_filenames)

In [ ]:
dataset = "ATAS_10" # ATAS, ATAS_5, ATAS_10

### Generate cellpose3d results

In [ ]:
model_dir = '../DATA/ATAS/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_04_28_14_46_49.933105'

In [ ]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename)) 
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/%s/cellpose3d/%s" % (dataset, test_filename), masks)

### Generate cellpose2d results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename)) 
    masks, _, _ = model.eval(list(img), do_3D=False, flow_threshold=flow_threshold, channels = [0,0])
    masks = cp_utils.stitch3D(np.array(masks))
    
    np.save("./results/%s/cellpose2d/%s" % (dataset, test_filename), masks)

### Generate cellstitch results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename))
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch/%s" % (dataset, test_filename), cellstitch) 

### Generate cellstitch3d results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename))
    
    cellstitch = np.load("./results/%s/cellpose3d/%s" % (dataset, test_filename))
    
    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)
    
    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch3d/%s" % (dataset, test_filename), cellstitch)

# Benchmark Results

In [ ]:
ap_threshold = 0.5
method = "cellpose2d"

In [ ]:
data = [] 

for filename in test_filenames:
    print("Starting %s" % filename)
    labels = np.load('../DATA/%s/labels/%s' % (dataset, filename))

    masks = np.load("./results/%s/%s/%s" % (dataset, method, filename))

    ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    row = [
        filename,
        ap, 
        tp, 
        fp, 
        fn, 
        precision,
        recall
    ]

    data.append(row)

df = pd.DataFrame(data, columns=[
    "filename", 
    "ap", 
    "tp", 
    "fp", 
    "fn",
    "precision",
    "recall"
])

df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)